# Install pytorch packagess

In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Install scipy and networkx packages

In [ ]:
!pip install 'scipy>=1.8'
!pip install 'networkx<2.7'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.8 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2
    Uninstalling networkx-3.2:
      Successfully uninstalled networkx-3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


# extract all

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


!!!
#read dataset from drive

In [ ]:
import torch
dataFromGraphFile = torch.load("/content/drive/MyDrive/master-project-github/Comparison-SkipGNN/Run_Model/Graph_Comparison_SkipGNN.pt")

#CustomLinkSplit Implementation

In [ ]:
import warnings
from copy import copy
from typing import List, Optional, Union

import torch
from torch import Tensor

from torch_geometric.data import Data, HeteroData
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.data.storage import EdgeStorage
from torch_geometric.transforms import BaseTransform
from torch_geometric.typing import EdgeType
from torch_geometric.utils import negative_sampling

class CustomLinkSplit(BaseTransform):
    r"""Performs an edge-level random split into training, validation and test
    sets of a :class:`~torch_geometric.data.Data` or a
    :class:`~torch_geometric.data.HeteroData` object
    (functional name: :obj:`random_link_split`).
    The split is performed such that the training split does not include edges
    in validation and test splits; and the validation split does not include
    edges in the test split.

    .. code-block::

        from torch_geometric.transforms import RandomLinkSplit

        transform = RandomLinkSplit(is_undirected=True)
        train_data, val_data, test_data = transform(data)

    Args:
        num_val (int or float, optional): The number of validation edges.
            If set to a floating-point value in :math:`[0, 1]`, it represents
            the ratio of edges to include in the validation set.
            (default: :obj:`0.1`)
        num_test (int or float, optional): The number of test edges.
            If set to a floating-point value in :math:`[0, 1]`, it represents
            the ratio of edges to include in the test set.
            (default: :obj:`0.2`)
        is_undirected (bool): If set to :obj:`True`, the graph is assumed to be
            undirected, and positive and negative samples will not leak
            (reverse) edge connectivity across different splits. Note that this
            only affects the graph split, label data will not be returned
            undirected.
            (default: :obj:`False`)
        key (str, optional): The name of the attribute holding
            ground-truth labels.
            If :obj:`data[key]` does not exist, it will be automatically
            created and represents a binary classification task
            (:obj:`1` = edge, :obj:`0` = no edge).
            If :obj:`data[key]` exists, it has to be a categorical label from
            :obj:`0` to :obj:`num_classes - 1`.
            After negative sampling, label :obj:`0` represents negative edges,
            and labels :obj:`1` to :obj:`num_classes` represent the labels of
            positive edges. (default: :obj:`"edge_label"`)
        split_labels (bool, optional): If set to :obj:`True`, will split
            positive and negative labels and save them in distinct attributes
            :obj:`"pos_edge_label"` and :obj:`"neg_edge_label"`, respectively.
            (default: :obj:`False`)
        add_negative_train_samples (bool, optional): Whether to add negative
            training samples for link prediction.
            If the model already performs negative sampling, then the option
            should be set to :obj:`False`.
            Otherwise, the added negative samples will be the same across
            training iterations unless negative sampling is performed again.
            (default: :obj:`True`)
        neg_sampling_ratio (float, optional): The ratio of sampled negative
            edges to the number of positive edges. (default: :obj:`1.0`)
        disjoint_train_ratio (int or float, optional): If set to a value
            greater than :obj:`0.0`, training edges will not be shared for
            message passing and supervision. Instead,
            :obj:`disjoint_train_ratio` edges are used as ground-truth labels
            for supervision during training. (default: :obj:`0.0`)
        edge_types (Tuple[EdgeType] or List[EdgeType], optional): The edge
            types used for performing edge-level splitting in case of
            operating on :class:`~torch_geometric.data.HeteroData` objects.
            (default: :obj:`None`)
        rev_edge_types (Tuple[EdgeType] or List[Tuple[EdgeType]], optional):
            The reverse edge types of :obj:`edge_types` in case of operating
            on :class:`~torch_geometric.data.HeteroData` objects.
            This will ensure that edges of the reverse direction will be
            split accordingly to prevent any data leakage.
            Can be :obj:`None` in case no reverse connection exists.
            (default: :obj:`None`)
    """
    def __init__(
        self,
        num_val: Union[int, float] = 0.1,
        num_test: Union[int, float] = 0.1,
        is_undirected: bool = False,
        key: str = 'edge_label',
        split_labels: bool = False,
        add_negative_train_samples: bool = True,
        neg_sampling_ratio: float = 1.0,
        disjoint_train_ratio: Union[int, float] = 0.0,
        edge_types: Optional[Union[EdgeType, List[EdgeType]]] = None,
        rev_edge_types: Optional[Union[EdgeType, List[EdgeType]]] = None,

    ):
        if isinstance(edge_types, list):
            if rev_edge_types is None:
                rev_edge_types = [None] * len(edge_types)

            assert isinstance(rev_edge_types, list)
            assert len(edge_types) == len(rev_edge_types)

        self.num_val = num_val
        self.num_test = num_test
        self.is_undirected = is_undirected
        self.key = key
        self.split_labels = split_labels
        self.add_negative_train_samples = add_negative_train_samples
        self.neg_sampling_ratio = neg_sampling_ratio
        self.disjoint_train_ratio = disjoint_train_ratio
        self.edge_types = edge_types
        self.rev_edge_types = rev_edge_types


    def setEdgesArray(self, train_edges,val_edges,test_edges,train_val_edges,additional_edge_label_index,additional_edge_label):
      self.train_edges = train_edges
      self.val_edges = val_edges
      self.test_edges = test_edges
      self.train_val_edges = train_val_edges
      self.additional_edge_label_index = additional_edge_label_index
      self.additional_edge_label = additional_edge_label

    def forward(
        self,
        data: Union[Data, HeteroData],
    ) -> Union[Data, HeteroData]:
        edge_types = self.edge_types
        rev_edge_types = self.rev_edge_types

        train_data, val_data, test_data = copy(data), copy(data), copy(data)

        if isinstance(data, HeteroData):
            if edge_types is None:
                raise ValueError(
                    "The 'RandomLinkSplit' transform expects 'edge_types' to "
                    "be specified when operating on 'HeteroData' objects")

            if not isinstance(edge_types, list):
                edge_types = [edge_types]
                rev_edge_types = [rev_edge_types]

            stores = [data[edge_type] for edge_type in edge_types]
            train_stores = [train_data[edge_type] for edge_type in edge_types]
            val_stores = [val_data[edge_type] for edge_type in edge_types]
            test_stores = [test_data[edge_type] for edge_type in edge_types]
        else:
            rev_edge_types = [None]
            stores = [data._store]
            train_stores = [train_data._store]
            val_stores = [val_data._store]
            test_stores = [test_data._store]

        for item in zip(stores, train_stores, val_stores, test_stores,
                        rev_edge_types):
            store, train_store, val_store, test_store, rev_edge_type = item

            is_undirected = self.is_undirected
            is_undirected &= not store.is_bipartite()
            is_undirected &= (rev_edge_type is None
                              or store._key == data[rev_edge_type]._key)

            edge_index = store.edge_index
            if is_undirected:
                mask = edge_index[0] <= edge_index[1]
                print("Mask : ",mask)
                print("Mask type: ",type(mask))
                print("Mask lenght : ",len(mask))
                perm = mask.nonzero(as_tuple=False).view(-1)
                print("Perm : ", perm)
                print("Perm type : ", type(perm))
                print("Perm lenght : ", len(perm))
                #perm = perm[torch.randperm(perm.size(0), device=perm.device)]
                print("Perm : ", perm)
                print("Perm lenght : ", len(perm))
            else:
                device = edge_index.device
                perm = torch.randperm(edge_index.size(1), device=device)

            num_val = len(self.val_edges)
            if isinstance(num_val, float):
                num_val = int(num_val * perm.numel())
            num_test = len(self.test_edges)
            if isinstance(num_test, float):
                num_test = int(num_test * perm.numel())

            num_train = len(self.train_edges)

            print("perm.numel()", perm.numel() ,"num_train", num_train ,"num_val", num_val, "num_test", num_test)

            if num_train <= 0:
                raise ValueError("Insufficient number of edges for training")
            """
            train_edges = perm[:num_train]
            val_edges = perm[num_train:num_train + num_val]
            test_edges = perm[num_train + num_val:]
            train_val_edges = perm[:num_train + num_val]
            """
            train_edges = self.train_edges
            val_edges = self.val_edges
            test_edges = self.test_edges
            sameNodesEdges = self.train_val_edges

            print("Train_edges:  ", train_edges)
            print("Train_edges lenght:  ", len(train_edges))
            print("Val_edges:  ", val_edges)
            print("Val_edges lenght:  ", len(val_edges))
            print("Test_edges: ", test_edges)
            print("Test_edges lenght: ", len(test_edges))
            print("train_val_edges: ", train_val_edges)
            print("train_val_edges lenght: ", len(train_val_edges))
            print("self disjoint_train_ratio", self.disjoint_train_ratio)
            num_disjoint = self.disjoint_train_ratio
            if isinstance(num_disjoint, float):
                num_disjoint = int(num_disjoint * train_edges.numel())
            if num_train - num_disjoint <= 0:
                raise ValueError("Insufficient number of edges for training")

            print("num_disjoint", num_disjoint)
            # Create data splits:
            self._split(train_store, train_edges, is_undirected,
                        rev_edge_type)
            self._split(val_store, val_edges, is_undirected, rev_edge_type)
            self._split(test_store, test_edges, is_undirected,
                        rev_edge_type)

            # Create negative samples:
            num_neg_train = 0
            if self.add_negative_train_samples:
                if num_disjoint > 0:
                    num_neg_train = int(num_disjoint * self.neg_sampling_ratio)
                else:
                    num_neg_train = int(num_train * self.neg_sampling_ratio)
            num_neg_val = int(num_val * self.neg_sampling_ratio)
            num_neg_test = int(num_test * self.neg_sampling_ratio)

            num_neg = num_neg_train + num_neg_val + num_neg_test

            print("num_neg_train", num_neg_train ,"num_neg_val", num_neg_val, "num_neg_test", num_neg_test)

            size = store.size()
            if store._key is None or store._key[0] == store._key[-1]:
                size = size[0]
            neg_edge_index = negative_sampling(edge_index, size,
                                               num_neg_samples=num_neg,
                                               method='sparse')

            # Adjust ratio if not enough negative edges exist
            if neg_edge_index.size(1) < num_neg:
                num_neg_found = neg_edge_index.size(1)
                ratio = num_neg_found / num_neg
                warnings.warn(
                    f"There are not enough negative edges to satisfy "
                    "the provided sampling ratio. The ratio will be "
                    f"adjusted to {ratio:.2f}.")
                num_neg_train = int((num_neg_train / num_neg) * num_neg_found)
                num_neg_val = int((num_neg_val / num_neg) * num_neg_found)
                num_neg_test = num_neg_found - num_neg_train - num_neg_val

            # Create labels:
            if num_disjoint > 0:
                train_edges = train_edges[:num_disjoint]
            self._create_label(
                store,
                train_edges,
                neg_edge_index[:, num_neg_val + num_neg_test:],
                out=train_store,
            )
            self._create_label(
                store,
                val_edges,
                neg_edge_index[:, :num_neg_val],
                out=val_store,
            )
            self._create_label(
                store,
                test_edges,
                neg_edge_index[:, num_neg_val:num_neg_val + num_neg_test],
                out=test_store,
            )

        return train_data, val_data, test_data

    def _split(
        self,
        store: EdgeStorage,
        index: Tensor,
        is_undirected: bool,
        rev_edge_type: EdgeType,
    ) -> EdgeStorage:

        edge_attrs = {key for key in store.keys() if store.is_edge_attr(key)}
        for key, value in store.items():
            if key == 'edge_index':
                continue

            if key == 'edge_nodes_attributes':
                continue

            if key in edge_attrs:
                value = value[index]
                if is_undirected:
                    value = torch.cat([value, value], dim=0)
                store[key] = value

        edge_index = self.additional_edge_label_index
        if is_undirected:
            edge_index = torch.cat([edge_index, edge_index.flip([0])], dim=-1)
        store.edge_index = edge_index

        if rev_edge_type is not None:
            rev_store = store._parent()[rev_edge_type]
            for key in rev_store.keys():
                if key not in store:
                    del rev_store[key]  # We delete all outdated attributes.
                elif key == 'edge_index':
                    rev_store.edge_index = store.edge_index.flip([0])
                else:
                    rev_store[key] = store[key]

        return store

    def _create_label(
        self,
        store: EdgeStorage,
        index: Tensor,
        neg_edge_index: Tensor,
        out: EdgeStorage,
    ) -> EdgeStorage:

        edge_index = store.edge_index[:, index]
        print("test1 : ",edge_index)
        print("test1 : ",len(edge_index))
        if hasattr(store, self.key):
            edge_label = store[self.key]
            edge_label = edge_label[index]
            # Increment labels by one. Note that there is no need to increment
            # in case no negative edges are added.
            if neg_edge_index.numel() > 0:
                assert edge_label.dtype == torch.long
                assert edge_label.size(0) == edge_index.size(1)
                edge_label.add_(1)
            if hasattr(out, self.key):
                delattr(out, self.key)
        else:
            edge_label = torch.ones(index.numel(), device=index.device)
        print("test2 : ",edge_label)
        print("test2 : ",len(edge_label))
        if neg_edge_index.numel() > 0:
            neg_edge_label = edge_label.new_zeros((neg_edge_index.size(1), ) +
                                                  edge_label.size()[1:])

        if self.split_labels:
            """
            edge_label = torch.cat([edge_label, self.additional_edge_label], dim=0)
            edge_index = torch.cat([edge_index, self.additional_edge_label_index], dim=-1)
            """
            out[f'pos_{self.key}'] = edge_label
            out[f'pos_{self.key}_index'] = edge_index
            if neg_edge_index.numel() > 0:
                out[f'neg_{self.key}'] = neg_edge_label
                out[f'neg_{self.key}_index'] = neg_edge_index

        else:
            if neg_edge_index.numel() > 0:
                edge_label = torch.cat([edge_label, neg_edge_label], dim=0)
                edge_index = torch.cat([edge_index, neg_edge_index], dim=-1)
                """
                edge_label = torch.cat([edge_label, self.additional_edge_label], dim=0)
                edge_index = torch.cat([edge_index, self.additional_edge_label_index], dim=-1)
                """
            out[self.key] = edge_label
            out[f'{self.key}_index'] = edge_index
        f.write("Out " + str(out)  +" \n")
        return out

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}(num_val={self.num_val}, '
                f'num_test={self.num_test})')

#init variables

In [ ]:
import numpy as np

# null numpy array
train_edges_list = []
test_edges_list = []
val_edges_list = []



#mask operation
mask = dataFromGraphFile.edge_index[0] <= dataFromGraphFile.edge_index[1]

#edgeIndexArray is built
edgeIndexArray = mask.nonzero(as_tuple=False).view(-1)

totalEdgeNumber = len(edgeIndexArray)

maxEdgeNumber = len(dataFromGraphFile.edge_index[0])

#init ratios
val_ratio = 0.1
test_ratio = 0.1
train_ratio = 1-val_ratio-test_ratio



#init index values
trainEdgeIndex = 0
valEdgeIndex = 0
testEdgeIndex = 0

#init index values
trainEdgeIndexGeneDisease = 0
valEdgeIndexGeneDisease = 0
testEdgeIndexGeneDisease = 0

#init index values
trainEdgeIndexGeneGene = 0
valEdgeIndexGeneGene = 0
testEdgeIndexGeneGene = 0

#init index values
trainEdgeIndexDiseaseDisease = 0
valEdgeIndexDiseaseDisease = 0
testEdgeIndexDiseaseDisease = 0



#build gene-disease index dict
geneDiseaseEdgeIndexDict = {}

#build gene id array
processedGeneId = []

# null numpy array
train_edges_list = []
test_edges_list = []
val_edges_list = []

# edge number between disease-gene or vice-versa

numberOfEdgeBetweenDiseaseGene = 0

numberOfEdgeBetweenGeneGene = 0
numberOfEdgeBetweenDiseaseDisease = 0

def initVariables():
  global edgeIndexArray
  global totalEdgeNumber
  global val_ratio
  global test_ratio
  global train_ratio
  global train_edges
  global test_edges
  global val_edges
  global train_val_edges
  global trainEdgeIndex
  global testEdgeIndex
  global valEdgeIndex

  global trainEdgeIndexGeneDisease
  global testEdgeIndexGeneDisease
  global valEdgeIndexGeneDisease

  global trainEdgeIndexGeneGene
  global testEdgeIndexGeneGene
  global valEdgeIndexGeneGene

  global trainEdgeIndexDiseaseDisease
  global testEdgeIndexDiseaseDisease
  global valEdgeIndexDiseaseDisease

  global maxTrainEdgeIndex
  global maxValEdgeIndex
  global maxTestEdgeIndex
  global geneDiseaseEdgeIndexDict

  global train_edges_numpy
  global test_edges_numpy
  global val_edges_numpy

  global train_edges_list
  global test_edges_list
  global val_edges_list

  global numberOfEdgeBetweenDiseaseGene
  global numberOfEdgeBetweenDiseaseDisease
  global numberOfEdgeBetweenGeneGene

  global maxEdgeNumber

  global edge_index_label_temp

  #mask operation
  mask = dataFromGraphFile.edge_index[0] <= dataFromGraphFile.edge_index[1]

  #edgeIndexArray is built
  edgeIndexArray = mask.nonzero(as_tuple=False).view(-1)

  totalEdgeNumber = len(edgeIndexArray)

  maxEdgeNumber = len(dataFromGraphFile.edge_index[0])

  #init ratios
  val_ratio = 0.1
  test_ratio = 0.1
  train_ratio = 1-val_ratio-test_ratio


  train_val_edges = torch.empty(1)
  train_edges = torch.empty(1)
  test_edges = torch.empty(1)
  val_edges = torch.empty(1)

  #init index values
  trainEdgeIndex = 0
  valEdgeIndex = 0
  testEdgeIndex = 0


  #build gene-disease index dict
  geneDiseaseEdgeIndexDict = {}

  #build gene id array
  processedGeneId = []

  # null numpy array
  train_edges_list = []
  test_edges_list = []
  val_edges_list = []

  #edg_label_index
  edge_index_label_temp = []

  numberOfEdgeBetweenDiseaseGene = 0
  numberOfEdgeBetweenGeneGene = 0
  numberOfEdgeBetweenDiseaseDisease = 0

  #init index values
  trainEdgeIndexGeneDisease = 0
  valEdgeIndexGeneDisease = 0
  testEdgeIndexGeneDisease = 0

  #init index values
  trainEdgeIndexGeneGene = 0
  valEdgeIndexGeneGene = 0
  testEdgeIndexGeneGene = 0

  #init index values
  trainEdgeIndexDiseaseDisease = 0
  valEdgeIndexDiseaseDisease = 0
  testEdgeIndexDiseaseDisease = 0



In [ ]:
def getNumberOfEdgesBetweenDiseaseGene():
  global numberOfEdgeBetweenDiseaseGene
  global maxTrainEdgeIndex
  global maxTestEdgeIndex
  global maxValEdgeIndex
  global train_ratio
  global test_ratio
  global val_ratio
  index = 0
  while(index<maxEdgeNumber):
    if(dataFromGraphFile.edge_index[0][index]<=dataFromGraphFile.edge_index[1][index]):
      firstNode = True if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[0][index]]!="") else False
      firstNodeID = "" if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[0][index]]=="") else dataFromGraphFile.id[dataFromGraphFile.edge_index[0][index]]
      # check second node is gene or not
      # if node is gene it is assigned to True, otherwise assigned False
      secondNode = True if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[1][index]]!="") else False
      secondNodeID = "" if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[1][index]]=="") else dataFromGraphFile.id[dataFromGraphFile.edge_index[1][index]]
      if(firstNode and not secondNode):
        numberOfEdgeBetweenDiseaseGene+=1
      elif(not firstNode and secondNode):
        numberOfEdgeBetweenDiseaseGene+=1
    index+=1
  maxTrainEdgeIndex = numberOfEdgeBetweenDiseaseGene * train_ratio
  maxTestEdgeIndex = numberOfEdgeBetweenDiseaseGene * test_ratio
  maxValEdgeIndex = numberOfEdgeBetweenDiseaseGene * val_ratio

def getNumberOfEdgesBetweenGeneGeneOrDiseaseDisease():
  global numberOfEdgeBetweenGeneGene
  global numberOfEdgeBetweenDiseaseDisease
  index = 0
  while(index<maxEdgeNumber):
    if(dataFromGraphFile.edge_index[0][index]<=dataFromGraphFile.edge_index[1][index]):
      firstNode = True if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[0][index]]!="") else False
      firstNodeID = "" if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[0][index]]=="") else dataFromGraphFile.id[dataFromGraphFile.edge_index[0][index]]
      # check second node is gene or not
      # if node is gene it is assigned to True, otherwise assigned False
      secondNode = True if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[1][index]]!="") else False
      secondNodeID = "" if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[1][index]]=="") else dataFromGraphFile.id[dataFromGraphFile.edge_index[1][index]]
      if(firstNode and  secondNode):
        numberOfEdgeBetweenGeneGene+=1
      elif(not firstNode and not secondNode):
        numberOfEdgeBetweenDiseaseDisease+=1
    index+=1

In [ ]:
"""
getNumberOfEdgesBetweenDiseaseGene()
print(numberOfEdgeBetweenDiseaseGene)
getNumberOfEdgesBetweenGeneGeneOrDiseaseDisease()
print(numberOfEdgeBetweenGeneGene)
print(numberOfEdgeBetweenDiseaseDisease)
print(mask)
print(edgeIndexArray[:60])

"""

'\ngetNumberOfEdgesBetweenDiseaseGene()\nprint(numberOfEdgeBetweenDiseaseGene)\ngetNumberOfEdgesBetweenGeneGeneOrDiseaseDisease()\nprint(numberOfEdgeBetweenGeneGene)\nprint(numberOfEdgeBetweenDiseaseDisease)\nprint(mask)\nprint(edgeIndexArray[:60])\n\n'

#read uniref_50

In [ ]:
# !
import csv
rowsUniref50 = []
with open("/content/drive/MyDrive/masterProject/GeneFeatures_Uniref50.csv", "r") as file:
    csvreader = csv.reader(file)
    headerUniref50 = next(csvreader)
    for row in csvreader:
        rowsUniref50.append(row)

In [ ]:
print(headerUniref50)
print(rowsUniref50[0])

['geneid', 'gene_symbol', 'uniprotid', 'gene_dsi', 'gene_dpi', 'gene_pli', 'protein_class', 'protein_class_name', 'sequence', 'lenght', 'molWeight', 'uniref50_members']
['1', 'A1BG', 'P04217', '0.7', '0.538', '4.9917e-09', 'DTO_05007575', 'Receptor', 'MSMLVVFLLLWGVTWGPVTEAAIFYETQPSLWAESESLLKPLANVTLTCQAHLETPDFQLFKNGVAQEPVHLDSPAIKHQFLLTGDTQGRYRCRSGLSTGWTQLSKLLELTGPKSLPAPWLSMAPVSWITPGLKTTAVCRGVLRGVTFLLRREGDHEFLEVPEAQEDVEATFPVHQPGNYSCSYRTDGEGALSEPSATVTIEELAAPPPPVLMHHGESSQVLHPGNKVTLTCVAPLSGVDFQLRRGEKELLVPRSSTSPDRIFFHLNAVALGDGGHYTCRYRLHDNQNGWSGDSAPVELILSDETLPAPEFSPEPESGRALRLRCLAPLEGARFALVREDRGGRRVHRFQSPAGTEALFELHNISVADSANYSCVYVDLKPPFGGSAPSERLELHVDGPPPRPQLRATWSGAVLAGRDAVLRCEGPIPDVTFELLREGETKAVKTVRTPGAAANLELIFVGPQHAGNYRCRYRSWVPHTFESELSDPVELLVAES', '495', '54254', 'P04217,P85980,V9HWD8,P04217-2,A0A2J8JM56,H2R119,H2P0G5,A0A2R9AU60,Q7Z3U3,A0A7J8CD38']


build uniref50 dict

In [ ]:
dictUniref50={}
for element in rowsUniref50:
  if(len(element)>11):
    dictUniref50[element[2]] = element[11]
  #else:
    #print("Problem in geneid : ", element[0])
print(dictUniref50.get("P04217"))

P04217,P85980,V9HWD8,P04217-2,A0A2J8JM56,H2R119,H2P0G5,A0A2R9AU60,Q7Z3U3,A0A7J8CD38


#build a dict that store gene-disease edges indexes

In [ ]:
def buildGeneDiseaseEdgeIndexDict():
  global maxEdgeNumber
  index = 0
  while(index<maxEdgeNumber):
    global geneDiseaseEdgeIndexDict

    # checn edge_index[0] is lower or equal with dataFromGraphFile.edge_index[1][index]
    if(dataFromGraphFile.edge_index[0][index]<=dataFromGraphFile.edge_index[1][index]):
      # check first node is gene or not
      # if node is gene it is assigned to True, otherwise assigned False
      firstNode = True if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[0][index]]!="") else False
      firstNodeID = "" if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[0][index]]=="") else dataFromGraphFile.id[dataFromGraphFile.edge_index[0][index]]
      # check second node is gene or not
      # if node is gene it is assigned to True, otherwise assigned False
      secondNode = True if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[1][index]]!="") else False
      secondNodeID = "" if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[1][index]]=="") else dataFromGraphFile.id[dataFromGraphFile.edge_index[1][index]]
      if(firstNode and not secondNode):
        if(geneDiseaseEdgeIndexDict.get(firstNodeID) != None and geneDiseaseEdgeIndexDict.get(firstNodeID) != "" and geneDiseaseEdgeIndexDict.get(firstNodeID) != []):
          tempList = geneDiseaseEdgeIndexDict.get(firstNodeID)
          tempList.append(index)
          geneDiseaseEdgeIndexDict[firstNodeID] = tempList
        else:
          tempList=[]
          tempList.append(index)
          geneDiseaseEdgeIndexDict[firstNodeID] = tempList
      elif(not firstNode and secondNode):
        if(geneDiseaseEdgeIndexDict.get(secondNodeID) != None and geneDiseaseEdgeIndexDict.get(secondNodeID) != "" and geneDiseaseEdgeIndexDict.get(secondNodeID) != []):
          tempList = geneDiseaseEdgeIndexDict.get(secondNodeID)
          tempList.append(index)
          geneDiseaseEdgeIndexDict[secondNodeID] = tempList
        else:
          tempList=[]
          tempList.append(index)
          geneDiseaseEdgeIndexDict[secondNodeID] = tempList
    index+=1


In [ ]:
buildGeneDiseaseEdgeIndexDict()

In [ ]:
for key, value in geneDiseaseEdgeIndexDict.items():
    print(key, ' : ', value)

Streaming output truncated to the last 5000 lines.
Q14204  :  [12716, 12878, 13119, 13762, 40899, 51380, 51518, 51657, 72632, 89027, 111167, 111186, 113024, 113076, 174055, 174056, 174058, 174059, 174061, 174063, 174066, 174067, 174068]
P54259  :  [12718, 13772, 128907, 137598, 137626, 137654, 137680, 137708, 137738]
Q14232  :  [12719, 13124, 34095, 64310, 155005, 185808]
Q92915  :  [12720, 12890, 13835, 62089, 145505, 145548, 207275]
Q16595  :  [12723, 12891, 13853, 19055, 29444, 213688, 213690, 213691]
O43424  :  [12725, 13916, 21027, 115711, 115788, 115832, 115880, 115932, 115980, 116029, 229406]
P06865  :  [12727, 13169, 13937, 34107, 51390, 51528, 51667, 239835, 239836, 239839, 239840, 239841, 239842, 239843, 239844, 239845]
Q09470  :  [12730, 12909, 13182, 13998, 50127, 241239, 275181, 275182, 275183, 275184, 275187]
Q14003  :  [12731, 12910, 14002, 74714, 275497]
Q9UK17  :  [12732, 14004, 21058, 75284, 123713, 123803, 123898, 123935, 275554, 275556, 275559]
P55157  :  [12736, 12

# functions using dict

In [ ]:

def inList(geneID):
  global processedGeneId
  if geneID in processedGeneId:
    return True
  else:
    return False

###################################################################################
def processGeneIdTrainFromDict(geneID):
  global trainEdgeIndex
  global maxTrainEdgeIndex
  global train_edges_list
  global geneDiseaseEdgeIndexDict
  global processedGeneId
  global numberOfEdgeBetweenDiseaseGene
  global trainEdgeIndexGeneDisease
  #print("processGeneIdTrain is start : ", geneID)
  # check geneId is processed or not
  if(not inList(geneID)):
    #geneid in genediseaseEdgeIndexdict
    if( geneID in geneDiseaseEdgeIndexDict.keys()):
      # check trainEdgeIndex maximum number
      if(trainEdgeIndex < maxTrainEdgeIndex + len(geneDiseaseEdgeIndexDict.get(geneID)) and trainEdgeIndexGeneDisease <numberOfEdgeBetweenDiseaseGene*train_ratio):
          train_edges_list.extend(geneDiseaseEdgeIndexDict.get(geneID))
          trainEdgeIndex += len(geneDiseaseEdgeIndexDict.get(geneID))
          trainEdgeIndexGeneDisease += len(geneDiseaseEdgeIndexDict.get(geneID))
          print("Train edge added : ", geneID," appended_len : ",len(geneDiseaseEdgeIndexDict.get(geneID))," trainEdgeIndexGeneDisease: ",trainEdgeIndexGeneDisease)

          #append geneID to processedGeneId list
          processedGeneId.append(geneID)

def processGeneIdTestFromDict(geneID):
  global testEdgeIndex
  global maxTestEdgeIndex
  global test_edges_list
  global geneDiseaseEdgeIndexDict
  global processedGeneId
  global numberOfEdgeBetweenDiseaseGene
  global testEdgeIndexGeneDisease
  #print("processGeneIdTest is start : ", geneID)
  # check geneId is processed or not
  if(not inList(geneID)):
    #geneid in genediseaseEdgeIndexdict
    if( geneID in geneDiseaseEdgeIndexDict.keys()):
      # check trainEdgeIndex maximum number
      if(testEdgeIndex<maxTestEdgeIndex + len(geneDiseaseEdgeIndexDict.get(geneID)) and testEdgeIndexGeneDisease<numberOfEdgeBetweenDiseaseGene*test_ratio):
          test_edges_list.extend(geneDiseaseEdgeIndexDict.get(geneID))
          testEdgeIndex += len(geneDiseaseEdgeIndexDict.get(geneID))
          testEdgeIndexGeneDisease += len(geneDiseaseEdgeIndexDict.get(geneID))
          print("Test edge added : ", geneID," appended_len : ",len(geneDiseaseEdgeIndexDict.get(geneID))," testEdgeIndexGeneDisease: ",testEdgeIndexGeneDisease)

          #append geneID to processedGeneId list
          processedGeneId.append(geneID)

def processGeneIdValFromDict(geneID):
  global valEdgeIndex
  global maxValEdgeIndex
  global val_edges_list
  global geneDiseaseEdgeIndexDict
  global processedGeneId
  global numberOfEdgeBetweenDiseaseGene
  global valEdgeIndexGeneDisease
  #print("processGeneIdVal is start : ", geneID)
  # check geneId is processed or not
  if(not inList(geneID)):
    #geneid in genediseaseEdgeIndexdict
    if( geneID in geneDiseaseEdgeIndexDict.keys()):
      # check trainEdgeIndex maximum number
      if(valEdgeIndex<maxValEdgeIndex + len(geneDiseaseEdgeIndexDict.get(geneID)) and valEdgeIndexGeneDisease<numberOfEdgeBetweenDiseaseGene*val_ratio):
          val_edges_list.extend(geneDiseaseEdgeIndexDict.get(geneID))
          valEdgeIndex += len(geneDiseaseEdgeIndexDict.get(geneID))
          valEdgeIndexGeneDisease +=len(geneDiseaseEdgeIndexDict.get(geneID))
          print("Val edge added : ", geneID," appended_len : ",len(geneDiseaseEdgeIndexDict.get(geneID))," valEdgeIndexGeneDisease: ",valEdgeIndexGeneDisease)


          #append geneID to processedGeneId list
          processedGeneId.append(geneID)


# split edge based on geneDiseaseEdgeIndexDict

In [ ]:
def splitEdgesBasedOnUniref50UsingDict():

  global edgeIndexArray
  global totalEdgeNumber
  global val_ratio
  global test_ratio
  global train_ratio
  global train_edges
  global test_edges
  global train_val_edges
  global val_edges
  global trainEdgeIndex
  global testEdgeIndex
  global valEdgeIndex

  global trainEdgeIndexGeneDisease
  global testEdgeIndexGeneDisease
  global valEdgeIndexGeneDisease

  global trainEdgeIndexGeneGene
  global testEdgeIndexGeneGene
  global valEdgeIndexGeneGene

  global trainEdgeIndexDiseaseDisease
  global testEdgeIndexDiseaseDisease
  global valEdgeIndexDiseaseDisease

  global maxTrainEdgeIndex
  global maxValEdgeIndex
  global maxTestEdgeIndex
  global geneDiseaseEdgeIndexDict

  global train_edges_numpy
  global test_edges_numpy
  global val_edges_numpy

  global train_edges_list
  global test_edges_list
  global val_edges_list

  global numberOfEdgeBetweenDiseaseGene
  global numberOfEdgeBetweenDiseaseDisease
  global numberOfEdgeBetweenGeneGene

  global maxEdgeNumber

  index=0
  maxTrainEdgeIndex = int(numberOfEdgeBetweenDiseaseGene * train_ratio)
  maxTestEdgeIndex = int(numberOfEdgeBetweenDiseaseGene * test_ratio)
  maxValEdgeIndex = int(numberOfEdgeBetweenDiseaseGene * val_ratio)

  while(index<maxEdgeNumber):
    """
    print(dataFromGraphFile.edge_index[0][index])
    print(dataFromGraphFile.edge_index[1][index])
    print(dataFromGraphFile.id[dataFromGraphFile.edge_index[0][index]])
    print(dataFromGraphFile.id[dataFromGraphFile.edge_index[1][index]])
    print(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[0][index]])
    print(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[1][index]])
    """
    # check dataFromGraphFile.edge_index[0][index] is less or equal with dataFromGraphFile.edge_index[1][index]
    if(dataFromGraphFile.edge_index[0][index]<=dataFromGraphFile.edge_index[1][index]):
      # check first node is gene or not
      # if node is gene it is assigned to True, otherwise assigned False
      firstNode = True if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[0][index]]!="") else False
      firstNodeID = "" if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[0][index]]=="") else dataFromGraphFile.id[dataFromGraphFile.edge_index[0][index]]
      # check second node is gene or not
      # if node is gene it is assigned to True, otherwise assigned False
      secondNode = True if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[1][index]]!="") else False
      secondNodeID = "" if(dataFromGraphFile.gene_smybol[dataFromGraphFile.edge_index[1][index]]=="") else dataFromGraphFile.id[dataFromGraphFile.edge_index[1][index]]


      # firstnode is gene and second node is disease, we want to gather this edge especially in test and val
      # firstNodeID is not null but, secondNodeID is null.
      if(firstNode and not secondNode):
        if(not inList(firstNodeID)):
          processGeneIdTestFromDict(firstNodeID)
          if(dictUniref50.get(firstNodeID)!="" and dictUniref50.get(firstNodeID)!=None):
            #print("Test uniret50 split :",dictUniref50.get(firstNodeID).split(","))
            for element in dictUniref50.get(firstNodeID).split(","):
              processGeneIdTestFromDict(element)
            print("Test list len : ", len(test_edges_list))

        if(not inList(firstNodeID)):
          processGeneIdValFromDict(firstNodeID)
          if(dictUniref50.get(firstNodeID)!="" and dictUniref50.get(firstNodeID)!=None):
            #print("Val uniret50 split :",dictUniref50.get(firstNodeID).split(","))
            for element in dictUniref50.get(firstNodeID).split(","):
              processGeneIdValFromDict(element)
            print("Val list len : ", len(val_edges_list))

        if(not inList(firstNodeID)):
          processGeneIdTrainFromDict(firstNodeID)
          if(dictUniref50.get(firstNodeID)!="" and dictUniref50.get(firstNodeID)!=None):
            #print("Train uniret50 split :",dictUniref50.get(firstNodeID).split(","))
            for element in dictUniref50.get(firstNodeID).split(","):
              processGeneIdTrainFromDict(element)
            print("Train list len : ", len(train_edges_list))


      # firstnode is disease and second node is gene, we want to gather this edge especially in val and test
      elif(not firstNode and secondNode):

        if(not inList(secondNodeID)):
          processGeneIdTestFromDict(secondNodeID)
          if(dictUniref50.get(secondNodeID)!="" and dictUniref50.get(secondNodeID)!=None):
            #print("Test uniret50 split :",dictUniref50.get(secondNodeID).split(","))
            for element in dictUniref50.get(secondNodeID).split(","):
              processGeneIdTestFromDict(element)
            print("Test list len : ", len(test_edges_list))

        if(not inList(secondNodeID)):
          processGeneIdValFromDict(secondNodeID)
          if(dictUniref50.get(secondNodeID)!="" and dictUniref50.get(secondNodeID)!=None):
            #print("Val uniret50 split :",dictUniref50.get(secondNodeID).split(","))
            for element in dictUniref50.get(secondNodeID).split(","):
              processGeneIdValFromDict(element)
            print("Val list len : ", len(val_edges_list))

        if(not inList(secondNodeID)):
          processGeneIdTrainFromDict(secondNodeID)
          if(dictUniref50.get(secondNodeID)!="" and dictUniref50.get(secondNodeID)!=None):
            #print("Train uniret50 split :",dictUniref50.get(secondNodeID).split(","))
            for element in dictUniref50.get(secondNodeID).split(","):
              processGeneIdTrainFromDict(element)
            print("Train list len : ", len(train_edges_list))

      # firstnode is gene and second node is gene, we want to gather this edge especially in train and val

      elif(firstNode and secondNode):
        edge_index_label_temp.append(index)
        """
        if(maxTrainEdgeIndex>trainEdgeIndex and trainEdgeIndexGeneGene<numberOfEdgeBetweenGeneGene*train_ratio):
          train_edges_list.append(index)
          trainEdgeIndex+=1
          trainEdgeIndexGeneGene+=1
        elif(maxValEdgeIndex>valEdgeIndex and valEdgeIndexGeneGene<numberOfEdgeBetweenGeneGene*val_ratio):
          val_edges_list.append(index)
          valEdgeIndex+=1
          valEdgeIndexGeneGene+=1
        elif(maxTestEdgeIndex>testEdgeIndex and testEdgeIndexGeneGene<numberOfEdgeBetweenGeneGene*test_ratio):
          test_edges_list.append(index)
          testEdgeIndex+=1
          testEdgeIndexGeneGene+=1
          """

      # firstnode is disease and second node is disease, we want to gather this edge especially in train and val
      elif(not firstNode and not secondNode):
        edge_index_label_temp.append(index)
        """
        if(maxTrainEdgeIndex>trainEdgeIndex and trainEdgeIndexDiseaseDisease<numberOfEdgeBetweenDiseaseDisease*train_ratio):
          train_edges_list.append(index)
          trainEdgeIndex+=1
          trainEdgeIndexDiseaseDisease+=1
        elif(maxValEdgeIndex>valEdgeIndex and valEdgeIndexDiseaseDisease<numberOfEdgeBetweenDiseaseDisease*val_ratio):
          val_edges_list.append(index)
          valEdgeIndex+=1
          valEdgeIndexDiseaseDisease+=1
        elif(maxTestEdgeIndex>testEdgeIndex and testEdgeIndexDiseaseDisease<numberOfEdgeBetweenDiseaseDisease*test_ratio):
          test_edges_list.append(index)
          testEdgeIndex+=1
          testEdgeIndexDiseaseDisease+=1
        """


    if(index%1000 == 0):
      print("index : ", index)
    index = index  +  1

  train_edges = torch.tensor(train_edges_list)
  test_edges = torch.tensor(test_edges_list)
  val_edges = torch.tensor(val_edges_list)


  train_val_edges = torch.tensor(train_edges_list + val_edges_list)

In [ ]:


def buildEdgeLabelIndex(edge_index_label_temp):
  index= 0
  edge_label_index2 = torch.zeros([2, len(edge_index_label_temp)], dtype=torch.int32)
  edge_label2 = torch.ones([len(edge_index_label_temp)], dtype=torch.int32)
  print(edge_label_index2)
  print(edge_label2)
  while(index<len(edge_index_label_temp)):
    edge_label_index2[0][index] = dataFromGraphFile.edge_index[0][edge_index_label_temp[index]]
    edge_label_index2[1][index] = dataFromGraphFile.edge_index[1][edge_index_label_temp[index]]
    index+=1
  return edge_label_index2, edge_label2

"""
edge_label_index2, edge_label2 = buildEdgeLabelIndex(edge_index_label_temp)
print(edge_label_index2)
print(edge_label2)
"""

'\nedge_label_index2, edge_label2 = buildEdgeLabelIndex(edge_index_label_temp)\nprint(edge_label_index2)\nprint(edge_label2)\n'

In [ ]:
def getEdgeLabelIndexFromIndex(indexList):
  edge_label_index = torch.zeros([2, len(indexList)], dtype=torch.int32)
  #edge_label = torch.ones([len(indexList)], dtype=torch.int32)
  index = 0
  while(index<len(indexList)):
    edge_label_index[0][index] = dataFromGraphFile.edge_index[0][indexList[index]]
    edge_label_index[1][index] = dataFromGraphFile.edge_index[1][indexList[index]]
    index+=1
  return edge_label_index

print(len(train_edges_list))
print(getEdgeLabelIndexFromIndex(train_edges_list))
print(len(getEdgeLabelIndexFromIndex(train_edges_list)[0]))

0
tensor([], size=(2, 0), dtype=torch.int32)
0


#KZClinkSplit


In [ ]:
from typing import Union, Tuple

import copy

import torch
from torch import Tensor

from torch_geometric.data import Data
from torch_geometric.utils import add_self_loops, negative_sampling
from torch_geometric.transforms import BaseTransform


class KZCLinkSplit(BaseTransform):
    r"""Performs an edge-level random split into training, validation and test
    sets.
    The split is performed such that the training split does not include edges
    in validation and test splits; and the validation split does not include
    edges in the test split.

    .. code-block::

        from torch_geometric.transforms import RandomLinkSplit

        transform = RandomLinkSplit(is_undirected=True)
        train_data, val_data, test_data = transform(data)

    Args:
        num_val (int or float, optional): The number of validation edges.
            If set to a floating-point value in :math:`[0, 1]`, it represents
            the ratio of edges to include in the validation set.
            (default: :obj:`0.1`)
        num_test (int or float, optional): The number of test edges.
            If set to a floating-point value in :math:`[0, 1]`, it represents
            the ratio of edges to include in the test set.
            (default: :obj:`0.2`)
        is_undirected (bool): If set to :obj:`True`, the graph is assumed to be
            undirected, and positive and negative samples will not leak
            (reverse) edge connectivity across different splits.
            (default: :obj:`False`)
        key (str, optional): The name of the attribute holding
            ground-truth labels.
            If :obj:`data[key]` does not exist, it will be automatically
            created and represents a binary classification task
            (:obj:`1` = edge, :obj:`0` = no edge).
            If :obj:`data[key]` exists, it has to be a categorical label from
            :obj:`0` to :obj:`num_classes - 1`.
            After negative sampling, label :obj:`0` represents negative edges,
            and labels :obj:`1` to :obj:`num_classes` represent the labels of
            positive edges. (default: :obj:`"edge_label"`)
        split_labels (bool, optional): If set to :obj:`True`, will split
            positive and negative labels and save them in distinct attributes
            :obj:`"pos_edge_label"` and :obj:`"neg_edge_label"`, respectively.
            (default: :obj:`False`)
        add_negative_train_samples (bool, optional): Whether to add negative
            training samples for link prediction.
            If the model already performs negative sampling, then the option
            should be set to :obj:`False`.
            Otherwise, the added negative samples will be the same across
            training iterations unless negative sampling is performed again.
            (default: :obj:`True`)
        neg_sampling_ratio: (float, optional): The ratio of sampled negative
            edges to the number of positive edges. (default: :obj:`1.0`)
    """
    def __init__(
        self,
        num_val: Union[int, float] = 0.1,
        num_test: Union[int, float] = 0.2,
        is_undirected: bool = False,
        key: str = 'edge_label',
        split_labels: bool = False,
        add_negative_train_samples: bool = True,
        neg_sampling_ratio: float = 1.0,
    ):
        self.num_val = num_val
        self.num_test = num_test
        self.is_undirected = is_undirected
        self.key = key
        self.split_labels = split_labels
        self.add_negative_train_samples = add_negative_train_samples
        self.neg_sampling_ratio = neg_sampling_ratio

    def setLists(self,trainEdgeLabelIndex, valEdgeLabelIndex, testEdgeLabelIndex):
      self.trainEdgeLabelIndex = trainEdgeLabelIndex
      self.valEdgeLabelIndex = valEdgeLabelIndex
      self.testEdgeLabelIndex = testEdgeLabelIndex


    def __call__(self, data: Data) -> Tuple[Data, Data, Data]:
        perm = torch.randperm(data.num_edges, device=data.edge_index.device)
        if self.is_undirected:
            perm = perm[data.edge_index[0] <= data.edge_index[1]]

        num_val, num_test = self.num_val, self.num_test
        if isinstance(num_val, float):
            num_val = int(num_val * perm.numel())
        if isinstance(num_test, float):
            num_test = int(num_test * perm.numel())

        num_train = perm.numel() - num_val - num_test
        if num_train <= 0:
            raise ValueError("Insufficient number of edges for training.")

        train_edges = perm[:num_train]
        val_edges = perm[num_train:num_train + num_val]
        test_edges = perm[num_train + num_val:]
        train_val_edges = perm[:num_train + num_val]

        # Create data splits:
        train_data = self._split_data(data, train_edges)
        val_data = self._split_data(data, train_edges)
        test_data = self._split_data(data, train_val_edges)

        # Create negative samples:
        num_neg_train = 0
        if self.add_negative_train_samples:
            num_neg_train = len(self.trainEdgeLabelIndex[0])
        num_neg_val = len(self.valEdgeLabelIndex[0])
        num_neg_test = len(self.testEdgeLabelIndex[0])

        num_neg = num_neg_train + num_neg_val + num_neg_test
        neg_edge_index = negative_sampling(
            add_self_loops(data.edge_index)[0], num_nodes=data.num_nodes,
            num_neg_samples=num_neg, method='sparse')

        # Create labels:
        self._create_label(
            data,
            train_edges,
            neg_edge_index[:, num_neg_val + num_neg_test:],
            self.trainEdgeLabelIndex,
            out=train_data,
        )
        self._create_label(
            data,
            val_edges,
            neg_edge_index[:, :num_neg_val],
            self.valEdgeLabelIndex,
            out=val_data,
        )
        self._create_label(
            data,
            test_edges,
            neg_edge_index[:, num_neg_val:num_neg_val + num_neg_test],
            self.testEdgeLabelIndex,
            out=test_data,
        )

        return train_data, val_data, test_data

    def _split(self, edge_index: Tensor, index: Tensor) -> Tensor:
        edge_index = edge_index[:, index]


        if self.is_undirected:
            edge_index = torch.cat([edge_index, edge_index.flip([0])], dim=-1)

        return edge_index

    def _split_data(self, data: Data, index: Tensor) -> Data:
        num_edges = data.num_edges

        data = copy.copy(data)
        data.edge_index = self._split(data.edge_index, index)

        for key, value in data.items():
          if key == 'edge_index':
            continue

          if key == 'edge_nodes_attributes':
            continue
            if isinstance(value, Tensor) and value.size(0) == num_edges:
                value = value[index]
                if self.is_undirected:
                    value = torch.cat([value, value], dim=0)
                data[key] = value

        return data

    def _create_label(self, data: Data, index: Tensor, neg_edge_index: Tensor,baseTensor ,
                      out: Data ):

        edge_index = baseTensor

        """
        if hasattr(data, self.key):
            edge_label = data[self.key]
            assert edge_label.dtype == torch.long and edge_label.dim() == 1
            edge_label = edge_label[index].add_(1)
            delattr(data, self.key)
        else:
            edge_label = torch.ones(index.numel(), device=index.device)
        """

        edge_label = torch.ones(len(baseTensor[0]), device=index.device)

        if neg_edge_index.numel() > 0:
            neg_edge_label = edge_label.new_zeros(neg_edge_index.size(1))

        if self.split_labels:
            out[f'pos_{self.key}'] = edge_label
            out[f'pos_{self.key}_index'] = edge_index
            if neg_edge_index.numel() > 0:
                out[f'neg_{self.key}'] = neg_edge_label
                out[f'neg_{self.key}_index'] = neg_edge_index

        else:
            if neg_edge_index.numel() > 0:
                edge_label = torch.cat([edge_label, neg_edge_label], dim=0)
                edge_index = torch.cat([edge_index, neg_edge_index], dim=-1)
            out[self.key] = edge_label
            out[f'{self.key}_index'] = edge_index

        return out

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}(num_val={self.num_val}, '
                f'num_test={self.num_test})')


#KZCLinkSplit run


In [ ]:
dataFromGraphFile.x = dataFromGraphFile.x.float()
import torchvision
from torchvision import datasets,transforms
import torchvision.transforms as transforms

initVariables()
getNumberOfEdgesBetweenDiseaseGene()
getNumberOfEdgesBetweenGeneGeneOrDiseaseDisease()
buildGeneDiseaseEdgeIndexDict()
print(geneDiseaseEdgeIndexDict)

transform = KZCLinkSplit(is_undirected=True, num_val=val_ratio,num_test=test_ratio,add_negative_train_samples=True)



splitEdgesBasedOnUniref50UsingDict()
transform.setLists(getEdgeLabelIndexFromIndex(train_edges_list), getEdgeLabelIndexFromIndex(val_edges_list), getEdgeLabelIndexFromIndex(test_edges_list))

train_data, val_data, test_data = transform(dataFromGraphFile)


Streaming output truncated to the last 5000 lines.
Train list len :  59119
Test list len :  7871
Val list len :  8095
Train edge added :  P60228  appended_len :  1  trainEdgeIndexGeneDisease:  59120
Train list len :  59120
Test list len :  7871
Val list len :  8095
Train edge added :  O43187  appended_len :  1  trainEdgeIndexGeneDisease:  59121
Train list len :  59121
Test list len :  7871
Val list len :  8095
Train edge added :  P52292  appended_len :  1  trainEdgeIndexGeneDisease:  59122
Train list len :  59122
Test list len :  7871
Val list len :  8095
Train edge added :  P00540  appended_len :  1  trainEdgeIndexGeneDisease:  59123
Train list len :  59123
Test list len :  7871
Val list len :  8095
Train edge added :  P62906  appended_len :  1  trainEdgeIndexGeneDisease:  59124
Train list len :  59124
Test list len :  7871
Val list len :  8095
Train edge added :  P46776  appended_len :  8  trainEdgeIndexGeneDisease:  59132
Train list len :  59132
Test list len :  7871
Val list len : 

In [ ]:
print(train_data)
print(test_data)
print(val_data)

Data(x=[19001, 1792], edge_index=[2, 604072], id=[19001], gene_smybol=[19001], edge_nodes_attributes=[753032], edge_label=[124262], edge_label_index=[2, 124262])
Data(x=[19001, 1792], edge_index=[2, 679580], id=[19001], gene_smybol=[19001], edge_nodes_attributes=[753032], edge_label=[15742], edge_label_index=[2, 15742])
Data(x=[19001, 1792], edge_index=[2, 604072], id=[19001], gene_smybol=[19001], edge_nodes_attributes=[753032], edge_label=[16190], edge_label_index=[2, 16190])


!!!
# outputs of the model that contains :

Val Accuracy,  Val Precision, Val Recall, Val F1


Test Accuracy,  Test Precison,  Test Recall, Test F1

In [ ]:
dataFromGraphFile.x = dataFromGraphFile.x.float()
from torch_geometric.transforms import RandomLinkSplit
import os.path as osp
from sklearn.model_selection import KFold

import torch
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
import numpy as np
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
import torchvision
from torchvision import datasets,transforms
import torchvision.transforms as transforms
from torch_geometric.utils import train_test_split_edges
from sklearn.metrics import roc_auc_score


#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
"""
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.1, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])

train_data, val_data, test_data = transform(dataFromGraphFile)
"""
"""
path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'Planetoid')
dataset = Planetoid(path, name='Cora', transform=transform)
"""



"""
from torch_geometric.transforms import RandomLinkSplit

transform = RandomLinkSplit(is_undirected=True, num_val=0.1,num_test=0.1,add_negative_train_samples=False)

#train_data, val_data, test_data = transform(dataset[0])

train_data, val_data, test_data = transform(dataFromGraphFile)
"""

# After applying the `RandomLinkSplit` transform, the data is transformed from
# a data object to a list of tuples (train_data, val_data, test_data), with
# each element representing the corresponding split.
"""
train_data, val_data, test_data = dataset[0]
"""


'\ntrain_data, val_data, test_data = dataset[0]\n'

# build networks, optimizers, loss functions , learning rate

# build hyper parameter tuning informations

In [ ]:

# 2 layer network
class Net1(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

# 3 layer network
class Net2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, middle_channels ,out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, middle_channels)
        self.conv3 = GCNConv(middle_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        return self.conv3(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

# 4 layer network
class Net3(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, middle1_channels , middle2_channels ,out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, middle1_channels)
        self.conv3 = GCNConv(middle1_channels, middle2_channels)
        self.conv4 = GCNConv(middle2_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = self.conv3(x, edge_index).relu()
        return self.conv4(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

# 5 layer network relu
class Net4(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, middle1_channels , middle2_channels ,middle3_channels ,out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, middle1_channels)
        self.conv3 = GCNConv(middle1_channels, middle2_channels)
        self.conv4 = GCNConv(middle2_channels, middle3_channels)
        self.conv5 = GCNConv(middle3_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = self.conv3(x, edge_index).relu()
        x = self.conv4(x, edge_index).relu()
        return self.conv5(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


        # 5 layer network relu
class Net5(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, middle1_channels , middle2_channels ,middle3_channels ,middle4_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, middle1_channels)
        self.conv3 = GCNConv(middle1_channels, middle2_channels)
        self.conv4 = GCNConv(middle2_channels, middle3_channels)
        self.conv5 = GCNConv(middle3_channels, middle4_channels)
        self.conv6 = GCNConv(middle4_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = self.conv3(x, edge_index).relu()
        x = self.conv4(x, edge_index).relu()
        x = self.conv5(x, edge_index).relu()
        return self.conv6(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


networks = [
    Net1(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/16), int(dataFromGraphFile.num_features/32)).to(device)
    ]

criterions = [
    torch.nn.L1Loss(),
    torch.nn.MSELoss(),
    torch.nn.CrossEntropyLoss(),
    torch.nn.CTCLoss(),
    torch.nn.NLLLoss(),
    torch.nn.PoissonNLLLoss(),
    torch.nn.GaussianNLLLoss(),
    torch.nn.KLDivLoss(),
    torch.nn.BCELoss(),
    torch.nn.BCEWithLogitsLoss(),
    torch.nn.MarginRankingLoss(),
    torch.nn.HingeEmbeddingLoss(),
    torch.nn.MultiLabelMarginLoss(),
    torch.nn.HuberLoss(),
    torch.nn.SmoothL1Loss(),
    torch.nn.SoftMarginLoss(),
    torch.nn.MultiLabelMarginLoss(),
    torch.nn.CosineEmbeddingLoss(),
    torch.nn.MultiMarginLoss(),
    torch.nn.TripletMarginLoss(),
    torch.nn.TripletMarginWithDistanceLoss()
]

elements = [
    [Net1(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/2), int(dataFromGraphFile.num_features/32)).to(device),torch.nn.PoissonNLLLoss()],
    [Net1(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/32)).to(device),torch.nn.PoissonNLLLoss()],
    [Net1(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/8), int(dataFromGraphFile.num_features/32)).to(device),torch.nn.PoissonNLLLoss()],
    [Net1(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/16), int(dataFromGraphFile.num_features/32)).to(device),torch.nn.PoissonNLLLoss()],
    [Net2(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/2), int(dataFromGraphFile.num_features/32),8).to(device),torch.nn.PoissonNLLLoss()],
    [Net2(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/32),8).to(device),torch.nn.PoissonNLLLoss()],
    [Net2(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/8), int(dataFromGraphFile.num_features/32),8).to(device),torch.nn.PoissonNLLLoss()],
    [Net2(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/16), int(dataFromGraphFile.num_features/32),8).to(device),torch.nn.PoissonNLLLoss()],
    [Net3(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/2), int(dataFromGraphFile.num_features/32),32,8).to(device),torch.nn.PoissonNLLLoss()],
    [Net3(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/32),32,8).to(device),torch.nn.PoissonNLLLoss()],
    [Net3(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/8), int(dataFromGraphFile.num_features/32),32,8).to(device),torch.nn.PoissonNLLLoss()],
    [Net3(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/16), int(dataFromGraphFile.num_features/32),32,16).to(device),torch.nn.PoissonNLLLoss()],
    [Net4(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/2), int(dataFromGraphFile.num_features/4),64,32,8).to(device),torch.nn.PoissonNLLLoss()],
    [Net4(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/2), int(dataFromGraphFile.num_features/16),64,32,8).to(device),torch.nn.PoissonNLLLoss()],
    [Net4(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/2), int(dataFromGraphFile.num_features/32),64,32,8).to(device),torch.nn.PoissonNLLLoss()],
    [Net4(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/4),64,32,8).to(device),torch.nn.PoissonNLLLoss()],
    [Net4(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/16),64,32,8).to(device),torch.nn.PoissonNLLLoss()],
    [Net4(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/32),64,32,8).to(device),torch.nn.PoissonNLLLoss()],
    [Net4(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/8), int(dataFromGraphFile.num_features/4),64,32,8).to(device),torch.nn.PoissonNLLLoss()],
    [Net4(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/8), int(dataFromGraphFile.num_features/16),64,32,8).to(device),torch.nn.PoissonNLLLoss()],
    [Net4(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/8), int(dataFromGraphFile.num_features/32),64,32,8).to(device),torch.nn.PoissonNLLLoss()]

]
"""
optimizers = [
        torch.optim.Adadelta(params=model.parameters()),
        torch.optim.Adagrad(params=model.parameters()),
        torch.optim.Adam(params=model.parameters()),
        torch.optim.AdamW(params=model.parameters()),
        torch.optim.SparseAdam(params=model.parameters()),
        torch.optim.Adamax(params=model.parameters()),
        torch.optim.ASGD(params=model.parameters()),
        torch.optim.LBFGS(params=model.parameters()),
        torch.optim.NAdam(params=model.parameters()),
        torch.optim.RAdam(params=model.parameters()),
        torch.optim.RMSprop(params=model.parameters()),
        torch.optim.Rprop(params=model.parameters()),
        torch.optim.SGD(params=model.parameters())

    ]
"""
"""
networks = [
    Net1(dataFromGraphFile.num_features, 128, 64).to(device),
    Net1(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/16)).to(device),
    Net1(dataFromGraphFile.num_features*4, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/16)).to(device),

    Net2(dataFromGraphFile.num_features, 128, 64, 16).to(device),
    Net2(dataFromGraphFile.num_features*2, int(dataFromGraphFile.num_features), int(dataFromGraphFile.num_features/2), int(dataFromGraphFile.num_features/4)).to(device),
    Net2(dataFromGraphFile.num_features*4, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/16), int(dataFromGraphFile.num_features/4)).to(device),

    Net3(dataFromGraphFile.num_features, 128, 64, 16, 4).to(device),
    Net3(dataFromGraphFile.num_features*4, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/16), int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/64)).to(device),
    Net3(dataFromGraphFile.num_features*16, dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/16), int(dataFromGraphFile.num_features/64)).to(device),

    Net4(dataFromGraphFile.num_features*4, 128, 64, 16, 16, 4).to(device),
    Net4(dataFromGraphFile.num_features*16, dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/8), int(dataFromGraphFile.num_features/16), 16).to(device),
    Net4(dataFromGraphFile.num_features*64, dataFromGraphFile.num_features*4, dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/2), int(dataFromGraphFile.num_features/8), 16).to(device),
    ]

optimizers = [
    torch.optim.Adam(params=model.parameters(), lr=0.01),
    torch.optim.Adam(params=model.parameters(), lr=0.1),
    torch.optim.Adam(params=model.parameters(), lr=0.001),

    torch.optim.Adadelta(params=model.parameters(), lr=0.001),
    torch.optim.Adadelta(params=model.parameters(), lr=0.01),
    torch.optim.Adadelta(params=model.parameters(), lr=0.1),

    torch.optim.Adamax(params=model.parameters(), lr=0.001),
    torch.optim.Adamax(params=model.parameters(), lr=0.01),
    torch.optim.Adamax(params=model.parameters(), lr=0.1),

    torch.optim.SDG(params=model.parameters(), lr=0.001),
    torch.optim.SDG(params=model.parameters(), lr=0.01),
    torch.optim.SDG(params=model.parameters(), lr=0.1)
]

criterions = [
    torch.nn.BCEWithLogitsLoss(),

    torch.nn.CrossEntropyLoss(),
    torch.nn.GaussianNLLLoss(),
    torch.nn.MultiLabelMarginLoss(),
    torch.nn.MultiMarginLoss()

]
"""



'\nnetworks = [\n    Net1(dataFromGraphFile.num_features, 128, 64).to(device),\n    Net1(dataFromGraphFile.num_features, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/16)).to(device),\n    Net1(dataFromGraphFile.num_features*4, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/16)).to(device),\n\n    Net2(dataFromGraphFile.num_features, 128, 64, 16).to(device),\n    Net2(dataFromGraphFile.num_features*2, int(dataFromGraphFile.num_features), int(dataFromGraphFile.num_features/2), int(dataFromGraphFile.num_features/4)).to(device),\n    Net2(dataFromGraphFile.num_features*4, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/16), int(dataFromGraphFile.num_features/4)).to(device),\n\n    Net3(dataFromGraphFile.num_features, 128, 64, 16, 4).to(device),\n    Net3(dataFromGraphFile.num_features*4, int(dataFromGraphFile.num_features/4), int(dataFromGraphFile.num_features/16), int(dataFromGraphFile.num_features/4), int(d

In [ ]:
class Model:

  def __init__(self,*parameters):

    self.setModel(parameters[0])
    self.setOptimizer(parameters[1])
    self.setLossFunction(parameters[2])

  def setModel(self,*modelParam):
    if(modelParam[0][0]=="Net1"):
      self.model = Net1(modelParam[0][1], modelParam[0][2], modelParam[0][3]).to(device)
    elif(modelParam[0][0]=="Net2"):
      self.model = Net2(modelParam[0][1], modelParam[0][2], modelParam[0][3], modelParam[0][4]).to(device)
    elif(modelParam[0][0]=="Net3"):
      self.model = Net3(modelParam[0][1], modelParam[0][2], modelParam[0][3], modelParam[0][4], modelParam[0][5]).to(device)
    elif(modelParam[0][0]=="Net4"):
      self.model = Net4(modelParam[0][1], modelParam[0][2], modelParam[0][3], modelParam[0][4], modelParam[0][5], modelParam[0][6]).to(device)
    elif(modelParam[0][0]=="Net5"):
      self.model = Net5(modelParam[0][1], modelParam[0][2], modelParam[0][3], modelParam[0][4], modelParam[0][5], modelParam[0][6], modelParam[0][7]).to(device)
    else:
      print("Not suitable format about Model")

  def setOptimizer(self,*optimizerParam):
    if(optimizerParam[0][0] == "Adadelta"):
      self.optimizer = torch.optim.Adadelta(params=self.model.parameters(),lr=optimizerParam[0][1])
    elif(optimizerParam[0][0] == "Adagrad"):
      self.optimizer = torch.optim.Adagrad(params=self.model.parameters(),lr=optimizerParam[0][1])
    elif(optimizerParam[0][0] == "Adam"):
      self.optimizer = torch.optim.Adam(params=self.model.parameters(),lr=optimizerParam[0][1])
    elif(optimizerParam[0][0] == "AdamW"):
      self.optimizer = torch.optim.AdamW(params=self.model.parameters(),lr=optimizerParam[0][1])
    elif(optimizerParam[0][0] == "SparseAdam"):
      self.optimizer = torch.optim.SparseAdam(params=self.model.parameters(),lr=optimizerParam[0][1])
    elif(optimizerParam[0][0] == "Adamax"):
      self.optimizer = torch.optim.Adamax(params=self.model.parameters(),lr=optimizerParam[0][1])
    elif(optimizerParam[0][0] == "ASGD"):
      self.optimizer = torch.optim.ASGD(params=self.model.parameters(),lr=optimizerParam[0][1])
    elif(optimizerParam[0][0] == "LBFGS"):
      self.optimizer = torch.optim.LBFGS(params=self.model.parameters(),lr=optimizerParam[0][1])
    elif(optimizerParam[0][0] == "NAdam"):
      self.optimizer = torch.optim.NAdam(params=self.model.parameters(),lr=optimizerParam[0][1])
    elif(optimizerParam[0][0] == "RAdam"):
      self.optimizer = torch.optim.RAdam(params=self.model.parameters(),lr=optimizerParam[0][1])
    elif(optimizerParam[0][0] == "RMSprop"):
      self.optimizer = torch.optim.RMSprop(params=self.model.parameters(),lr=optimizerParam[0][1])
    elif(optimizerParam[0][0] == "Rprop"):
      self.optimizer = torch.optim.Rprop(params=self.model.parameters(),lr=optimizerParam[0][1])
    elif(optimizerParam[0][0] == "SGD"):
      self.optimizer = torch.optim.SGD(params=self.model.parameters(),lr=optimizerParam[0][1])
    else:
      print("Not suitable format about optimizer")

  def setLossFunction(self,lossFuncName:str):
    if(lossFuncName == "L1Loss"):
      self.lossFunc =  torch.nn.L1Loss()
    elif(lossFuncName == "MSELoss"):
      self.lossFunc =  torch.nn.MSELoss()
    elif(lossFuncName == "CrossEntropyLoss"):
      self.lossFunc =  torch.nn.CrossEntropyLoss()
    elif(lossFuncName == "CTCLoss"):
      self.lossFunc =  torch.nn.CTCLoss()
    elif(lossFuncName == "NLLLoss"):
      self.lossFunc =  torch.nn.NLLLoss()
    elif(lossFuncName == "PoissonNLLLoss"):
      self.lossFunc =  torch.nn.PoissonNLLLoss()
    elif(lossFuncName == "GaussianNLLLoss"):
      self.lossFunc =  torch.nn.GaussianNLLLoss()
    elif(lossFuncName == "KLDivLoss"):
      self.lossFunc =  torch.nn.KLDivLoss()
    elif(lossFuncName == "BCELoss"):
      self.lossFunc =  torch.nn.BCELoss()
    elif(lossFuncName == "BCEWithLogitsLoss"):
      self.lossFunc =  torch.nn.BCEWithLogitsLoss()
    elif(lossFuncName == "MarginRankingLoss"):
      self.lossFunc =  torch.nn.MarginRankingLoss()
    elif(lossFuncName == "HingeEmbeddingLoss"):
      self.lossFunc =  torch.nn.HingeEmbeddingLoss()
    elif(lossFuncName == "MultiLabelMarginLoss"):
      self.lossFunc =  torch.nn.MultiLabelMarginLoss()
    elif(lossFuncName == "HuberLoss"):
      self.lossFunc =  torch.nn.HuberLoss()
    elif(lossFuncName == "SmoothL1Loss"):
      self.lossFunc =  torch.nn.SmoothL1Loss()
    elif(lossFuncName == "SoftMarginLoss"):
      self.lossFunc =  torch.nn.SoftMarginLoss()
    elif(lossFuncName == "CosineEmbeddingLoss"):
      self.lossFunc =  torch.nn.CosineEmbeddingLoss()
    elif(lossFuncName == "MultiMarginLoss"):
      self.lossFunc =  torch.nn.MultiMarginLoss()
    elif(lossFuncName == "TripletMarginLoss"):
      self.lossFunc =  torch.nn.TripletMarginLoss()
    elif(lossFuncName == "TripletMarginWithDistanceLoss"):
      self.lossFunc =  torch.nn.TripletMarginWithDistanceLoss()
    else:
      print("Not suitable format about loss function")

  def getModel(self):
    return self.model

  def getOptimizer(self):
    return self.optimizer

  def getLossFunc(self):
    return self.lossFunc


# model array

In [ ]:
model = [
    Model(["Net1",1792,112,28], ["AdamW",0.001], "BCEWithLogitsLoss")
]


# run hyper parameter tuning

In [ ]:
index = 0
max_test_accuracy=0
for element in model:
  try:
    prev_test_accuracy = 1
    model = element.getModel()
    criterion = element.getLossFunc()
    optimizer = element.getOptimizer()

    print("Working \n", "Model: ",model,  "\nLoss function: ",criterion, "\nOptimizer: ",optimizer)

    def train():
        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index)

        """We perform a new round of negative sampling for every training epoch:
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)
        """

        out = model.decode(z, train_data.edge_label_index).view(-1)
        loss = criterion(out, train_data.edge_label)
        loss.backward()
        optimizer.step()
        return loss


    @torch.no_grad()
    def testAccuracyScore(data):
        model.eval()
        z = model.encode(data.x, data.edge_index)
        out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
        """
        print("data.edge_label: ",  data.edge_label)
        print("Out: ", out)
        print("Data Edge Label", data.edge_label.cpu().numpy())
        print("Old Output ", out.cpu().numpy())
        print("New Output ", np.around(out.cpu().numpy()))
        """
        #return accuracy_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
        return accuracy_score(data.edge_label.cpu().numpy(), np.around(out.cpu().numpy()))

    @torch.no_grad()
    def testF1Score(data):
        model.eval()
        z = model.encode(data.x, data.edge_index)
        out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
        """
        print("data.edge_label: ",  data.edge_label)
        print("Out: ", out)
        print("Data Edge Label", data.edge_label.cpu().numpy())
        print("Old Output ", out.cpu().numpy())
        print("New Output ", np.around(out.cpu().numpy()))
        """
        #return accuracy_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
        return f1_score(data.edge_label.cpu().numpy(), np.around(out.cpu().numpy()))

    @torch.no_grad()
    def testPrecisionScore(data):
        model.eval()
        z = model.encode(data.x, data.edge_index)
        out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
        """
        print("data.edge_label: ",  data.edge_label)
        print("Out: ", out)
        print("Data Edge Label", data.edge_label.cpu().numpy())
        print("Old Output ", out.cpu().numpy())
        print("New Output ", np.around(out.cpu().numpy()))
        """
        #return accuracy_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
        return precision_score(data.edge_label.cpu().numpy(), np.around(out.cpu().numpy()))

    @torch.no_grad()
    def testRecallScore(data):
        model.eval()
        z = model.encode(data.x, data.edge_index)
        out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
        """
        print("data.edge_label: ",  data.edge_label)
        print("Out: ", out)
        print("Data Edge Label", data.edge_label.cpu().numpy())
        print("Old Output ", out.cpu().numpy())
        print("New Output ", np.around(out.cpu().numpy()))
        """
        #return accuracy_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
        return recall_score(data.edge_label.cpu().numpy(), np.around(out.cpu().numpy()))

    @torch.no_grad()
    def testSpecificityScore(data):
        model.eval()
        z = model.encode(data.x, data.edge_index)
        out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
        """
        print("data.edge_label: ",  data.edge_label)
        print("Out: ", out)
        print("Data Edge Label", data.edge_label.cpu().numpy())
        print("Old Output ", out.cpu().numpy())
        print("New Output ", np.around(out.cpu().numpy()))
        """
        #return accuracy_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
        return recall_score(np.logical_not(data.edge_label.cpu().numpy()), np.logical_not(np.around(out.cpu().numpy())))





    def train_epoch(model,device,dataloader,loss_fn,optimizer):
        train_loss,train_correct=0.0,0
        model.train()
        for images, labels in dataloader:

            images,labels = images.to(device),labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = loss_fn(output,labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            scores, predictions = torch.max(output.data, 1)
            train_correct += (predictions == labels).sum().item()

        return train_loss,train_correct

    def valid_epoch(model,device,dataloader,loss_fn):
      valid_loss, val_correct = 0.0, 0
      model.eval()
      with torch.no_grad():
        for images, labels in dataloader:

            images,labels = images.to(device),labels.to(device)
            output = model(images)
            loss=loss_fn(output,labels)
            valid_loss+=loss.item()*images.size(0)
            scores, predictions = torch.max(output.data,1)
            val_correct+=(predictions == labels).sum().item()

      return valid_loss,val_correct

    def test_epoch(model,device,dataloader,loss_fn):
      test_loss, test_correct = 0.0, 0
      model.eval()
      with torch.no_grad():
        for images, labels in dataloader:

            images,labels = images.to(device),labels.to(device)
            output = model(images)
            loss=loss_fn(output,labels)
            test_loss+=loss.item()*images.size(0)
            scores, predictions = torch.max(output.data,1)
            val_correct+=(predictions == labels).sum().item()

      return test_loss,test_correct

    @torch.no_grad()
    def testRocAuch(data):
        model.eval()
        z = model.encode(data.x, data.edge_index)
        out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
        return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())



    best_val_auc = final_test_auc = 0
    for epoch in range(1, 501):
      try:
        loss = train()
        val_accuracy = testAccuracyScore(val_data)
        test_accuracy = testAccuracyScore(test_data)
        val_precision = testPrecisionScore(val_data)
        test_precision = testPrecisionScore(test_data)
        val_recall = testRecallScore(val_data)
        test_recall = testRecallScore(test_data)
        #val_f1_score = test_f1_score(val_data)
        #test_f1_score = test_f1_score(test_data)
        val_roc_auc_score = testRocAuch(val_data)
        test_roc_auc_score = testRocAuch(test_data)
        val_specifity_score = testSpecificityScore(val_data)
        test_specifity_score = testSpecificityScore(test_data)
        if(max_test_accuracy<test_accuracy):
          max_test_accuracy = test_accuracy
        if epoch % 10 == 0 or prev_test_accuracy>test_accuracy:
          prev_test_accuracy = test_accuracy
          print("Epoch : ",epoch,
              "\n Val Accuracy: ", val_accuracy, "Val F1: ", (2*val_recall*val_precision)/(val_recall+val_precision), " Val Precision: ",val_precision, "Val Recall : ",val_recall, "Val Roc_Auc : ",val_roc_auc_score," Val Specifity : ",val_specifity_score,
              "\n Test Accuracy: ", test_accuracy,  " Test F1: ", (2*test_recall*test_precision)/(test_recall+test_precision), " Test Precison: ",test_precision, " Test Recall: ", test_recall,  "Test Roc_Auc : ",test_roc_auc_score, " Test Specifity : ",test_specifity_score)
      except:
        print("this eopch has a problem, epoch : ", epoch)

    print("Max test accuracy : ", max_test_accuracy)
  except:
    print("in this try we had a problem, please check upper informations")


Working 
 Model:  Net1(
  (conv1): GCNConv(1792, 112)
  (conv2): GCNConv(112, 28)
) 
Loss function:  BCEWithLogitsLoss() 
Optimizer:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)
Epoch :  1 
 Val Accuracy:  0.5002470660901791 Val F1:  0.6667764919072526  Val Precision:  0.5001235635734586 Val Recall :  1.0 Val Roc_Auc :  0.7857223039714081  Val Specifity :  0.0004941321803582458 
 Test Accuracy:  0.5000635243298183  Test F1:  0.6666949008978484  Test Precison:  0.5000317641827076  Test Recall:  1.0 Test Roc_Auc :  0.7536022072085676  Test Specifity :  0.00012704865963664082
Epoch :  2 
 Val Accuracy:  0.5 Val F1:  0.6666666666666666  Val Precision:  0.5 Val Recall :  1.0 Val Roc_Auc :  0.8180341993490671  Val Specifity :  0.0 
 Test Accuracy:  0.5  Test F1:  0.6666666666666666  Test Precison:  0.5  Test Recall:  